In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display

# Folder where uploads will be saved
UPLOAD_DIR = "uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

upload_btn = widgets.FileUpload(accept='.txt', multiple=False)
output = widgets.Output()

def on_upload_change(change):
    with output:
        output.clear_output()
        if upload_btn.value:
            file_info = upload_btn.value[0]
            filename = file_info['name']
            content = file_info['content']
            file_path = os.path.join(UPLOAD_DIR, filename)
            with open(file_path, 'wb') as f:
                f.write(content)
            global uploaded_file_path
            uploaded_file_path = file_path
            print(f" File uploaded: {filename}")
            print(f" Saved at: {uploaded_file_path}")

upload_btn.observe(on_upload_change, names='value')
display(widgets.VBox([upload_btn, output]))


In [2]:
import hashlib

def verify_uploaded_file(file_path, uploaded_widget):
    uploaded_bytes = uploaded_widget.value[0]['content']
    with open(file_path, 'rb') as f:
        saved_bytes = f.read()
    if hashlib.sha256(uploaded_bytes).hexdigest() == hashlib.sha256(saved_bytes).hexdigest():
        print(f" Verified OK: {file_path}")
    else:
        print(" File mismatch!")

verify_uploaded_file(uploaded_file_path, upload_btn)


 Verified OK: uploaded_files\C432.txt


In [3]:
# D-Algebra PODEM 
import os, time
from collections import deque

VERBOSE = True   # Set False to reduce printouts

#locate netlist -> txt file needed.
if 'uploaded_file_path' in globals():
    NETLIST_PATH = uploaded_file_path
else:
    fallback = "uploaded_files/C17.txt" #-> default netlist
    if os.path.exists(fallback):
        NETLIST_PATH = fallback
    else:
        NETLIST_PATH = None

if NETLIST_PATH is None:
    raise FileNotFoundError("Netlist file not found....")


def parse_netlist_from_file(path):
    inputs, outputs, gates = [], [], {}
    with open(path, 'r') as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith('#'): 
                continue
            if line.startswith("INPUT("):
                inputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif line.startswith("OUTPUT("):
                outputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif "=" in line:
                left, right = line.split("=",1)
                gate_name = left.strip()
                typ = right.strip().split("(",1)[0].strip().lower()
                args = right.strip().split("(",1)[1].rsplit(")",1)[0]
                ins = [a.strip() for a in args.split(",") if a.strip()!=""]
                gates[gate_name] = {"type": typ, "inputs": ins}
    return inputs, outputs, gates

# 5-D-algebra helpers (0,1,X,D,D_BAR)
## NOT gate
def logic_not(a):
    return {'0':'1','1':'0','X':'X','D':'D_BAR','D_BAR':'D'}.get(a,'X')

## AND gate
def logic_and_pair(a,b):
    if a=='0' or b=='0': return '0'
    if a=='1': return b
    if b=='1': return a
    if a=='X' or b=='X':
        # if one is X and other is D or D_BAR -> X 
        return 'X'
    if a==b: return a
    # D & D_BAR = 0
    if set([a,b])==set(['D','D_BAR']): return '0'
    return 'X'

## OR gate
def logic_or_pair(a,b):
    if a=='1' or b=='1': return '1'
    if a=='0': return b
    if b=='0': return a
    if a=='X' or b=='X': return 'X'
    if a==b: return a
    if set([a,b])==set(['D','D_BAR']): return '1'
    return 'X'

def eval_gate_D(gtype, ins_values):
    g = gtype.lower()
    if g in ('and','nand'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_and_pair(val, v)
        return logic_not(val) if g=='nand' else val #if nand then invert final output
    if g in ('or','nor'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_or_pair(val, v)
        return logic_not(val) if g=='nor' else val
    if g == 'not':
        return logic_not(ins_values[0])
    if g == 'buf':
        return ins_values[0]
    if g in ('xor'):
        # support 2-input xor well with D algebra
        if len(ins_values) != 2:
            return 'X'
        a,b = ins_values
        if 'X' in (a,b):
            return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = ag ^ bg
        bad  = af ^ bf
        if good==bad:
            return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    if g in ('xnor'):
        # support 2-input xor well with D algebra
        if len(ins_values) != 2:
            return 'X'
        a,b = ins_values
        if 'X' in (a,b):
            return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = 1-(ag ^ bg)
        bad  = 1-(af ^ bf)
        if good==bad:
            return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    return 'X'

# --- forward simulation with immediate fault injection and propagation ---
def simulate_with_fault(PIs, gates, assign, fault_node=None, fault_type=None):
    vals = {pi: assign.get(pi,'X') for pi in PIs}

    # inject PI fault
    if fault_node in PIs:
        good_val = vals[fault_node]
        if fault_type == 'sa0' and good_val == '1':
            vals[fault_node] = 'D'
        elif fault_type == 'sa1' and good_val == '0':
            vals[fault_node] = 'D_BAR'

    changed = True
    while changed:
        changed = False
        for g, info in gates.items():
            ins = [vals.get(inp, 'X') for inp in info['inputs']]
            outv = eval_gate_D(info['type'], ins)

            # inject fault for gate outputs (existing logic)
            if g == fault_node and fault_type is not None:
                good_val = outv
                if fault_type == 'sa0' and good_val == '1':
                    outv = 'D'
                elif fault_type == 'sa1' and good_val == '0':
                    outv = 'D_BAR'

            prev = vals.get(g, 'X')
            if prev != outv:
                vals[g] = outv
                changed = True

    return vals


# --- helpers to choose non-controlling input when backtracing ---
def non_controlling_value_for_gate(gtype):
    g = gtype.lower()
    # For AND/NAND non-controlling is '1' (i.e., to avoid driving output to 0)
    # For OR/NOR non-controlling is '0'
    if g in ('and','nand'): return '1'
    if g in ('or','nor'):   return '0'
    if g in ('not','buf','xor','xnor'): return None
    return None

# Backtrace: 
    '''
        - find primary input nodes with X values.
        - find primary input nodes with non controling values.
        - pick the first input node.
    '''
def backtrace_to_pi(node, desired_val, PIs, gates, current_vals):
    # If node is PI, done
    if node in PIs:
        return node, desired_val
    gate = gates[node]
    gtype = gate['type']
    inv = gtype in ('nand','nor','not')
    # choose an input to backtrace:
    # prefer inputs that are X or those that won't block propagation (non-controlling)
    pref = None
    # choose inputs that are X first
    for inp in gate['inputs']:
        if current_vals.get(inp,'X') == 'X':
            pref = inp; break
    if pref is None:
        # choose non-controlling input if any
        ncv = non_controlling_value_for_gate(gtype)
        if ncv is not None:
            for inp in gate['inputs']:
                if current_vals.get(inp,'X') != ncv:
                    pref = inp
                    break
    if pref is None:
        pref = gate['inputs'][0]
    next_val = logic_not(desired_val) if inv else desired_val
    return backtrace_to_pi(pref, next_val, PIs, gates, current_vals)

# --- Objective function: activation or propagation target ---
def objective(fault_node, fault_type, PIs, POs, gates, current_vals):
    # activation objective
    v_fault = current_vals.get(fault_node, 'X')
    if v_fault in ('X',) or v_fault not in ('D','D_BAR'):
        desired = '1' if fault_type == 'sa0' else '0'
        return fault_node, desired

    # propagation objective: choose a gate output (candidate) that is X and has D on some input
    for g, info in gates.items():
        outv = current_vals.get(g, 'X')
        if outv == 'X':
            in_vals = [ current_vals.get(inp,'X') for inp in info['inputs'] ]
            if any(iv in ('D','D_BAR') for iv in in_vals):
                # prefer gating where other inputs can be set to non-controlling
                return g, '1'  # desired value heuristic (chosen to allow backtrace)
    # as fallback, if a PO already shows D/D_BAR then objective satisfied
    for po in POs:
        if current_vals.get(po) in ('D','D_BAR'):
            return None, None
    return None, None

# --- implication helper (apply assignment + simulate) ---
def imply(PIs, gates, assign, fault_node, fault_type):
    return simulate_with_fault(PIs, gates, assign, fault_node, fault_type)

# --- PODEM recursive search ---
def podem(PIs, gates, POs, fault_node, fault_type, verbose=False, max_depth=200):
    # initial assign: all X
    init_assign = {pi:'X' for pi in PIs}
    visited = set()

    def assign_key(assign):
        return tuple((pi, assign[pi]) for pi in sorted(assign.keys()))

    def search(assign, depth=0):
        if depth > max_depth:
            return None
        vals = imply(PIs, gates, assign, fault_node, fault_type)
        # if verbose:
        #     print("  " * depth + "Sim vals:", vals)
        # check if fault propagated to any PO
        if any(vals.get(po) in ('D','D_BAR') for po in POs):
            return assign
        # if all PI assigned and no propagation -> fail
        if all(assign[pi] in ('0','1') for pi in PIs):
            return None

        key = assign_key(assign)
        if key in visited:
            return None
        visited.add(key)

        obj_node, obj_val = objective(fault_node, fault_type, PIs, POs, gates, vals)
        if obj_node is None:
            return None

        pi, pi_val = backtrace_to_pi(obj_node, obj_val, PIs, gates, vals)
        # if verbose:
        #     print("  " * depth + f"Objective: {obj_node}->{obj_val}, Backtrace: {pi}->{pi_val}")

        # If PI already assigned conflicting value, we should try the other value or backtrack
        if assign[pi] in ('0','1'):
            # nothing to try here - continue search deeper
            return search(assign, depth+1)

        # Try desired value first (heuristic), then opposite
        trial_order = [pi_val, '0' if pi_val=='1' else '1']
        for tv in trial_order:
            new_assign = assign.copy()
            new_assign[pi] = tv
            # if verbose:
            #     print("  " * depth + f"Try {pi}={tv}")
            res = search(new_assign, depth+1)
            if res is not None:
                return res
        return None

    return search(init_assign, depth=0)

# --- run PODEM for a single fault nicely ---
def run_podem_for_fault_true(netlist_path, fault_node, fault_type, verbose=VERBOSE):
    PIs, POs, GATES = parse_netlist_from_file(netlist_path)
    # if verbose:
        # print("PIs:", PIs)
        # print("POs:", POs)
        # print("Gates:", [(g,GATES[g]['type'],GATES[g]['inputs']) for g in GATES])
        # print(f"\nRunning PODEM for {fault_node} {fault_type}...\n")
    vec = podem(PIs, GATES, POs, fault_node, fault_type, verbose=verbose)
    if vec:
        # keep Xs as don't care in output
        print(f"\n Test cube for {fault_node} {fault_type}:")
        for pi in PIs:
            print(f"  {pi}: {vec.get(pi,'X')}")
        return True, vec
    else:
        print(f"\n Fault {fault_node} {fault_type} is untestable (or not found).")
        return False, None

# --- run all faults and produce coverage table ---
import pandas as pd
def run_all_faults_true(netlist_path, verbose=False, export_csv=True):
    PIs, POs, GATES = parse_netlist_from_file(netlist_path)
    results = []
    total = len(GATES) * 2
    detected = 0
    start = time.time()
    for g in GATES:
        for t in ('sa0','sa1'):
            if verbose:
                print(f"\n--- Fault: {g} {t} ---")
            ok, vec = run_podem_for_fault_true(netlist_path, g, t, verbose=False)
            if ok:
                detected += 1
                vec_str = " ".join(f"{pi}={vec.get(pi,'X')}" for pi in PIs)
            else:
                vec_str = "-"
            results.append({"Gate":g, "Fault_Type":t, "Detected":"Yes" if ok else "No", "Test_Cube":vec_str})
    elapsed = time.time() - start
    df = pd.DataFrame(results)
    coverage = (detected / total) * 100
    print(f"\nFault Coverage: {detected}/{total} = {coverage:.2f}%, Time: {elapsed:.2f}s")
    if export_csv:
        out = "podem_true_fault_coverage.csv"
        df.to_csv(out, index=False)
        print("Exported:", os.path.abspath(out))
    return df


print("Using netlist:", NETLIST_PATH)
PIs, POs, GATES = parse_netlist_from_file(NETLIST_PATH)
# print("PIs:", PIs)
# print("POs:", POs)
# print("Gates:", list(GATES.keys()))
# print("\nPODEM on a sample fault)
# _ok, _vec = run_podem_for_fault_true(NETLIST_PATH, "B", "sa1", verbose=VERBOSE)



Using netlist: uploaded_files\C432.txt


In [4]:
# for i in GATES.keys():
#     _ok, _vec = run_podem_for_fault_true(NETLIST_PATH, i, "sa0", verbose=VERBOSE)
#     _ok, _vec = run_podem_for_fault_true(NETLIST_PATH, i, "sa1", verbose=VERBOSE)

In [62]:
# GATES.keys()

# Algorithm - 1

In [4]:
import time
import pandas as pd

fault_results = []
total_start = time.time() 
PIs, POs, GATES = parse_netlist_from_file(uploaded_file_path)
fault_list = list(GATES.keys()) + list(PIs)

print("\n========== Running PODEM for All Faults ==========\n")

for gate_name in fault_list:

    start = time.time()
    ok0, vec0 = run_podem_for_fault_true(NETLIST_PATH, gate_name, "sa0", verbose=VERBOSE)
    end = time.time()
    t0 = round(end - start, 6)

    if ok0 and vec0:
        vec_str0 = " ".join([f"{pi}={vec0.get(pi, 'X')}" for pi in PIs])
    else:
        vec_str0 = "-"

    fault_results.append({
        "Fault_Node": gate_name,
        "Fault_Type": "sa0",
        "Detected": "YES" if ok0 else "NO",
        "Test_Cube": vec_str0,
        "Solve_Time_s": t0
    })

    start = time.time()
    ok1, vec1 = run_podem_for_fault_true(NETLIST_PATH, gate_name, "sa1", verbose=VERBOSE)
    end = time.time()
    t1 = round(end - start, 6)

    if ok1 and vec1:
        vec_str1 = " ".join([f"{pi}={vec1.get(pi, 'X')}" for pi in PIs])
    else:
        vec_str1 = "-"

    fault_results.append({
        "Fault_Node": gate_name,
        "Fault_Type": "sa1",
        "Detected": "YES" if ok1 else "NO",
        "Test_Cube": vec_str1,
        "Solve_Time_s": t1
    })

total_end = time.time()
total_time = round(total_end - total_start, 4)

df = pd.DataFrame(fault_results)

print("\n========== PODEM FAULT SUMMARY ==========")
print(df)

total_faults = len(df)
detected = df["Detected"].value_counts().get("YES", 0)
coverage = (detected / total_faults) * 100

print("\n------------ SUMMARY -------------")
print(f"Total Faults     : {total_faults}")
print(f"Detected Faults  : {detected}")
print(f"Undetected Faults: {total_faults - detected}")
print(f"Fault Coverage   : {coverage:.2f}%")
print(f"Total ATPG Time  : {total_time} seconds")
print("-----------------------------------")

net_name = os.path.splitext(os.path.basename(NETLIST_PATH))[0]
csv_file = f"{net_name}_podem_results.csv"
df.to_csv(csv_file, index=False)

print(f"\nResults saved to: {csv_file}")



========== Running PODEM for All Faults ==========


 Test cube for G10gat sa0:
  G1gat: 0
  G2gat: 1
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G10gat sa1:
  G1gat: 1
  G2gat: 1
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G11gat sa0:
  G1gat: X
  G2gat: 0
  G3gat: 0
  G6gat: X
  G7gat: 1

 Test cube for G11gat sa1:
  G1gat: X
  G2gat: 0
  G3gat: 1
  G6gat: 1
  G7gat: 1

 Test cube for G16gat sa0:
  G1gat: 1
  G2gat: 0
  G3gat: 1
  G6gat: 0
  G7gat: 0

 Test cube for G16gat sa1:
  G1gat: X
  G2gat: 1
  G3gat: 0
  G6gat: X
  G7gat: X

 Test cube for G19gat sa0:
  G1gat: X
  G2gat: X
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G19gat sa1:
  G1gat: X
  G2gat: 0
  G3gat: 0
  G6gat: X
  G7gat: 1

 Test cube for G22gat sa0:
  G1gat: 1
  G2gat: X
  G3gat: 1
  G6gat: X
  G7gat: X

 Test cube for G22gat sa1:
  G1gat: 0
  G2gat: 0
  G3gat: X
  G6gat: X
  G7gat: X

 Test cube for G23gat sa0:
  G1gat: X
  G2gat: 1
  G3gat: 0
  G6gat: X
  G7gat: X

 Test cube for G23gat sa1:
  G1ga

In [5]:
df = pd.read_csv("ex1_podem_results.csv")

In [6]:
df.head(100)

,Fault_Node,Fault_Type,Detected,Test_Cube,Solve_Time_s
0,e,sa0,YES,A=1 B=1 C=0 D=1,0.001000
1,e,sa1,YES,A=1 B=0 C=0 D=1,0.000000
2,f,sa0,YES,A=1 B=X C=1 D=1,0.000000
3,f,sa1,YES,A=1 B=0 C=0 D=1,0.001001
4,g,sa0,NO,-,0.000509
5,g,sa1,YES,A=1 B=X C=1 D=0,0.000000
6,h,sa0,YES,A=1 B=X C=1 D=1,0.000000
7,h,sa1,YES,A=0 B=X C=0 D=X,0.001008
8,k,sa0,YES,A=1 B=1 C=0 D=1,0.000000
9,k,sa1,YES,A=1 B=X C=1 D=0,0.000000


# Algorithm 2

In [7]:
# Optimized D-Algebra PODEM (PODEM semantics preserved)
import os, time
from collections import deque, defaultdict

VERBOSE = True   # Set False to reduce printouts

# --- locate netlist (same as before) ---
if 'uploaded_file_path' in globals():
    NETLIST_PATH = uploaded_file_path
else:
    fallback = "uploaded_files/C17.txt"
    if os.path.exists(fallback):
        NETLIST_PATH = fallback
    else:
        NETLIST_PATH = None

if NETLIST_PATH is None:
    raise FileNotFoundError("Netlist file not found. Upload and set 'uploaded_file_path' or place file at uploaded_files/C17.txt")

# --- Parser (same format as before) ---
def parse_netlist_from_file(path):
    inputs, outputs, gates = [], [], {}
    with open(path, 'r') as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith('#'):
                continue
            if line.startswith("INPUT("):
                inputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif line.startswith("OUTPUT("):
                outputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif "=" in line:
                left, right = line.split("=",1)
                gate_name = left.strip()
                typ = right.strip().split("(",1)[0].strip().lower()
                args = right.strip().split("(",1)[1].rsplit(")",1)[0]
                ins = [a.strip() for a in args.split(",") if a.strip()!=""]
                gates[gate_name] = {"type": typ, "inputs": ins}
    return inputs, outputs, gates

# --- D-algebra helpers (unchanged) ---
def logic_not(a):
    return {'0':'1','1':'0','X':'X','D':'D_BAR','D_BAR':'D'}.get(a,'X')

def logic_and_pair(a,b):
    if a=='0' or b=='0': return '0'
    if a=='1': return b
    if b=='1': return a
    if a=='X' or b=='X':
        return 'X'
    if a==b: return a
    if set([a,b])==set(['D','D_BAR']): return '0'
    return 'X'

def logic_or_pair(a,b):
    if a=='1' or b=='1': return '1'
    if a=='0': return b
    if b=='0': return a
    if a=='X' or b=='X': return 'X'
    if a==b: return a
    if set([a,b])==set(['D','D_BAR']): return '1'
    return 'X'

def eval_gate_D(gtype, ins_values):
    g = gtype.lower()
    if g in ('and','nand'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_and_pair(val, v)
        return logic_not(val) if g=='nand' else val
    if g in ('or','nor'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_or_pair(val, v)
        return logic_not(val) if g=='nor' else val
    if g == 'not':
        return logic_not(ins_values[0])
    if g == 'buf':
        return ins_values[0]
    if g == 'xor':
        if len(ins_values) != 2: return 'X'
        a,b = ins_values
        if 'X' in (a,b): return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = ag ^ bg
        bad  = af ^ bf
        if good==bad:
            return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    if g == 'xnor':
        if len(ins_values) != 2: return 'X'
        a,b = ins_values
        if 'X' in (a,b): return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = 1 - (ag ^ bg)
        bad  = 1 - (af ^ bf)
        if good==bad:
            return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    return 'X'

# --- Topological sort & fanout build for levelized simulation & x-path checks ---
def build_topology(PIs, POs, GATES):
    # build dependency graph (node inputs point to gate)
    indeg = {}
    children = defaultdict(list)  # fanout mapping: node -> list of gates that take it as input
    nodes = set(PIs) | set(GATES.keys())
    for g, info in GATES.items():
        indeg[g] = indeg.get(g, 0)
        for inp in info['inputs']:
            children[inp].append(g)
            # ensure input nodes exist in nodes set
            nodes.add(inp)
            indeg[g] = indeg.get(g, 0) + (0 if inp in PIs else 0)  # irrelevant but safe

    # We produce a topological order by simple levelization: repeatedly evaluate gates whose inputs are all PIs or other gates we've seen.
    # For correct levelization, compute actual indegree in terms of gate inputs that are gates.
    gate_indeg = {}
    for g, info in GATES.items():
        gate_indeg[g] = sum(1 for inp in info['inputs'] if inp in GATES)

    queue = deque([g for g in GATES if gate_indeg[g] == 0])
    topo = []
    while queue:
        x = queue.popleft()
        topo.append(x)
        # for every gate that takes x as input, reduce indeg
        for ch in children.get(x, []):
            if ch in gate_indeg:
                gate_indeg[ch] -= 1
                if gate_indeg[ch] == 0:
                    queue.append(ch)
    # If topo doesn't include all gates, append remaining deterministically (avoid missing nodes)
    if len(topo) < len(GATES):
        for g in GATES:
            if g not in topo:
                topo.append(g)
    # fanout mapping (including PIs)
    fanout = children  # node -> list of gates (or empty)
    return topo, fanout

# --- forward simulation using topological order + immediate fault injection ---
def simulate_with_fault_topo(PIs, POs, GATES, topo_order, assign, fault_node=None, fault_type=None):
    # assign: dict of PI -> '0'/'1'/'X'
    vals = {pi: assign.get(pi,'X') for pi in PIs}

    # inject PI fault if fault at PI
    if fault_node in PIs and fault_type is not None:
        good_val = vals.get(fault_node,'X')
        if fault_type == 'sa0' and good_val == '1':
            vals[fault_node] = 'D'
        elif fault_type == 'sa1' and good_val == '0':
            vals[fault_node] = 'D_BAR'

    # evaluate gates in topo order (one pass)
    for g in topo_order:
        info = GATES[g]
        ins = [vals.get(inp, 'X') for inp in info['inputs']]
        outv = eval_gate_D(info['type'], ins)
        # inject stuck-at on gate output if applicable
        if g == fault_node and fault_type is not None:
            good_val = outv
            if fault_type == 'sa0' and good_val == '1':
                outv = 'D'
            elif fault_type == 'sa1' and good_val == '0':
                outv = 'D_BAR'
        vals[g] = outv

    # Some circuits with reconvergent paths may need another pass to settle X/D interactions.
    # Do a small fixed number of relaxation iterations (usually 1 extra pass is enough).
    # But to stay safe we iterate until stable but with a small cap (e.g., 3 passes)
    for _ in range(2):
        changed = False
        for g in topo_order:
            info = GATES[g]
            ins = [vals.get(inp, 'X') for inp in info['inputs']]
            outv = eval_gate_D(info['type'], ins)
            if g == fault_node and fault_type is not None:
                good_val = outv
                if fault_type == 'sa0' and good_val == '1':
                    outv = 'D'
                elif fault_type == 'sa1' and good_val == '0':
                    outv = 'D_BAR'
            if vals.get(g,'X') != outv:
                vals[g] = outv
                changed = True
        if not changed:
            break

    return vals

# --- helpers for backtrace selection & observability path check (X-path) ---
def x_path_exists_from(node, current_vals, fanout, GATES, POs):
    # DFS from node forward to any PO following gates whose output currently is 'X' or directly to PO.
    # Conservative check: allow passing through nodes whose current value is 'X' or nodes that are PO themselves (even if not X).
    visited = set()
    stack = list(fanout.get(node, []))  # start with gates that take node as input
    while stack:
        g = stack.pop()
        if g in visited:
            continue
        visited.add(g)
        # If this gate is a PO (its name equals a PO label), check
        if g in POs:
            return True
        outv = current_vals.get(g, 'X')
        if outv == 'X':
            # If gate output is X, continue exploring its fanout
            if any(ch in POs for ch in fanout.get(g, [])):
                return True
            stack.extend(fanout.get(g, []))
        else:
            # if gate output is D/D_BAR and that gate is a PO -> success
            if g in POs and outv in ('D','D_BAR'):
                return True
            # else don't traverse through non-X outputs (conservative)
    return False

def non_controlling_value_for_gate(gtype):
    g = gtype.lower()
    if g in ('and','nand'): return '1'
    if g in ('or','nor'):   return '0'
    if g in ('not','buf','xor','xnor'): return None
    return None

def backtrace_to_pi_improved(node, desired_val, PIs, GATES, current_vals, fanout, POs):
    # If node is PI, done
    if node in PIs:
        return node, desired_val
    gate = GATES[node]
    gtype = gate['type']
    inv = gtype in ('nand','nor','not')
    pref = None

    # 1) prefer inputs that are X AND have an X-path to PO (so propagation possible)
    for inp in gate['inputs']:
        if current_vals.get(inp,'X') == 'X':
            # check x-path from this input (if it can reach a PO)
            if x_path_exists_from(inp, current_vals, fanout, GATES, POs):
                pref = inp
                break
    # 2) if none above, prefer any X input
    if pref is None:
        for inp in gate['inputs']:
            if current_vals.get(inp,'X') == 'X':
                pref = inp; break

    # 3) prefer a non-controlling input that doesn't currently hold the non-controlling value (so we can set it to non-controlling)
    if pref is None:
        ncv = non_controlling_value_for_gate(gtype)
        if ncv is not None:
            for inp in gate['inputs']:
                if current_vals.get(inp,'X') != ncv:
                    pref = inp
                    break

    # 4) fallback: first input
    if pref is None:
        pref = gate['inputs'][0]

    next_val = logic_not(desired_val) if inv else desired_val
    return backtrace_to_pi_improved(pref, next_val, PIs, GATES, current_vals, fanout, POs)

# --- objective with X-path preference ---
def objective_improved(fault_node, fault_type, PIs, POs, GATES, current_vals, fanout):
    # activation objective if not activated
    v_fault = current_vals.get(fault_node, 'X')
    if v_fault not in ('D','D_BAR'):
        desired = '1' if fault_type == 'sa0' else '0'
        return fault_node, desired

    # propagation objective: choose a gate output that is X and has D on some input,
    # but prefer one that has an X-path to a PO
    candidates = []
    for g, info in GATES.items():
        outv = current_vals.get(g, 'X')
        if outv == 'X':
            in_vals = [ current_vals.get(inp,'X') for inp in info['inputs'] ]
            if any(iv in ('D','D_BAR') for iv in in_vals):
                candidates.append(g)
    # prefer candidate with x-path
    for c in candidates:
        if x_path_exists_from(c, current_vals, fanout, GATES, POs):
            return c, '1'
    if candidates:
        return candidates[0], '1'
    # as fallback, if a PO already shows D/D_BAR then objective satisfied
    for po in POs:
        if current_vals.get(po) in ('D','D_BAR'):
            return None, None
    return None, None

# --- implication helper with memoization ---
def imply_with_cache(PIs, POs, GATES, topo_order, assign, fault_node, fault_type, sim_cache):
    # key: only assigned PIs (non-X)
    key = tuple(sorted((pi, val) for pi, val in assign.items() if val != 'X'))
    if key in sim_cache:
        return sim_cache[key]
    vals = simulate_with_fault_topo(PIs, POs, GATES, topo_order, assign, fault_node, fault_type)
    sim_cache[key] = vals
    return vals

# --- PODEM search (optimized) ---
def podem_optimized(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=False, max_depth=500):
    init_assign = {pi:'X' for pi in PIs}
    visited = set()
    sim_cache = {}  # memoize simulation results

    def assign_key(assign):
        # key based only on assigned PIs (non-X) -> smaller state space and more reuse
        return tuple(sorted((pi, assign[pi]) for pi in assign if assign[pi] != 'X'))

    def search(assign, depth=0):
        if depth > max_depth:
            return None
        vals = imply_with_cache(PIs, POs, GATES, topo_order, assign, fault_node, fault_type, sim_cache)
        # success?
        if any(vals.get(po) in ('D','D_BAR') for po in POs):
            return assign
        # if all PIs assigned & not propagated -> fail
        if all(assign[pi] in ('0','1') for pi in PIs):
            return None

        key = assign_key(assign)
        if key in visited:
            return None
        visited.add(key)

        obj_node, obj_val = objective_improved(fault_node, fault_type, PIs, POs, GATES, vals, fanout)
        if obj_node is None:
            return None

        pi, pi_val = backtrace_to_pi_improved(obj_node, obj_val, PIs, GATES, vals, fanout, POs)

        # if PI already assigned conflicting val -> try other or backtrack
        if assign[pi] in ('0','1'):
            return search(assign, depth+1)

        trial_order = [pi_val, '0' if pi_val=='1' else '1']
        for tv in trial_order:
            new_assign = assign.copy()
            new_assign[pi] = tv
            res = search(new_assign, depth+1)
            if res is not None:
                return res
        return None

    return search(init_assign, depth=0)

# --- run PODEM for a single fault, using parsed netlist & topology (no reparse) ---
def run_podem_for_fault_fast(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=VERBOSE):
    vec = podem_optimized(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=verbose)
    if vec:
        print(f"\n Test cube for {fault_node} {fault_type}:")
        for pi in PIs:
            print(f"  {pi}: {vec.get(pi,'X')}")
        return True, vec
    else:
        print(f"\n Fault {fault_node} {fault_type} is untestable (or not found).")
        return False, None

# --- run all faults with optimized code, produce CSV & time logs ---
import pandas as pd
import os, time
import pandas as pd

def run_all_faults_optimized(netlist_path, verbose=False, export_csv=True):
    # Parse once
    PIs, POs, GATES = parse_netlist_from_file(netlist_path)

    # Build topology once
    topo_order, fanout = build_topology(PIs, POs, GATES)

    # Build fault site list: include gate outputs + primary inputs (unique)
    fault_sites = list(GATES.keys())[:]  # gate outputs
    # add PIs that are not duplicated as gate outputs
    for pi in PIs:
        if pi not in fault_sites:
            fault_sites.append(pi)

    total = len(fault_sites) * 2
    results = []
    detected = 0
    start_all = time.time()

    print("\n======= Running Optimized PODEM for All Faults (including PIs) =======\n")

    for node in fault_sites:
        for t in ('sa0','sa1'):
            if verbose:
                print(f"\n--- Fault: {node} {t} ---")
            st = time.time()
            ok, vec = run_podem_for_fault_fast(
                PIs, POs, GATES,
                topo_order, fanout,
                node, t,
                verbose=verbose
            )
            et = time.time()

            # build printable PI vector using the PIs list (keeps order)
            if ok and vec:
                vec_str = " ".join(f"{pi}={vec.get(pi, 'X')}" for pi in PIs)
            else:
                vec_str = "-"

            if ok:
                detected += 1

            results.append({
                "Fault_Node": node,
                "Fault_Type": t,
                "Detected": "YES" if ok else "NO",
                "Test_Cube": vec_str,
                "Solve_Time_s": round(et - st, 6)
            })

    elapsed = time.time() - start_all
    df = pd.DataFrame(results)
    coverage = (detected / total) * 100 if total>0 else 0.0

    print(f"\nFault Coverage: {detected}/{total} = {coverage:.2f}%")
    print(f"Total ATPG Time: {elapsed:.2f}s")

    # dynamic CSV name from netlist path
    if export_csv:
        base = os.path.basename(netlist_path)
        name = os.path.splitext(base)[0]
        out_csv = f"{name}_podem_algo2.csv"
        df.to_csv(out_csv, index=False)
        print("Saved:", out_csv)

    return df


# ---------------- Example run ----------------
print("Using netlist:", NETLIST_PATH)
PIs, POs, GATES = parse_netlist_from_file(NETLIST_PATH)
topo_order, fanout = build_topology(PIs, POs, GATES)
print("Topological order (first 10 gates):", topo_order[:10])
print("\nNow running optimized PODEM on a sample fault (G180gat sa0) with VERBOSE=", VERBOSE)
# _ok, _vec = run_podem_for_fault_fast(PIs, POs, GATES, topo_order, fanout, "G180gat", "sa0", verbose=VERBOSE)

# To run full coverage:



Using netlist: uploaded_files\C17.txt
Topological order (first 10 gates): ['G10gat', 'G11gat', 'G16gat', 'G19gat', 'G22gat', 'G23gat']

Now running optimized PODEM on a sample fault (G180gat sa0) with VERBOSE= True


In [8]:
df_cov = run_all_faults_optimized(NETLIST_PATH, verbose=False, export_csv=True)


======= Running Optimized PODEM for All Faults (including PIs) =======


 Test cube for G10gat sa0:
  G1gat: 0
  G2gat: 1
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G10gat sa1:
  G1gat: 1
  G2gat: 1
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G11gat sa0:
  G1gat: X
  G2gat: 0
  G3gat: 0
  G6gat: X
  G7gat: 1

 Test cube for G11gat sa1:
  G1gat: X
  G2gat: 0
  G3gat: 1
  G6gat: 1
  G7gat: 1

 Test cube for G16gat sa0:
  G1gat: 1
  G2gat: 0
  G3gat: 1
  G6gat: 0
  G7gat: 0

 Test cube for G16gat sa1:
  G1gat: X
  G2gat: 1
  G3gat: 0
  G6gat: X
  G7gat: X

 Test cube for G19gat sa0:
  G1gat: X
  G2gat: X
  G3gat: 1
  G6gat: 1
  G7gat: X

 Test cube for G19gat sa1:
  G1gat: X
  G2gat: 0
  G3gat: 0
  G6gat: X
  G7gat: 1

 Test cube for G22gat sa0:
  G1gat: 1
  G2gat: X
  G3gat: 1
  G6gat: X
  G7gat: X

 Test cube for G22gat sa1:
  G1gat: 0
  G2gat: 0
  G3gat: X
  G6gat: X
  G7gat: X

 Test cube for G23gat sa0:
  G1gat: X
  G2gat: 1
  G3gat: 0
  G6gat: X
  G7gat: X

 Test cube fo

# Algorithm - 3 (Parallel processing)

In [5]:
from multiprocessing import Pool, cpu_count
import pandas as pd
import time, os

# Wrapper to compute a single fault → result dict
def solve_single_fault(args):
    (PIs, POs, GATES, topo_order, fanout, node, fault_type, verbose) = args
    st = time.time()
    ok, vec = run_podem_for_fault_fast(PIs, POs, GATES, topo_order, fanout, node, fault_type, verbose)
    et = time.time()

    if ok and vec:
        vec_str = " ".join(f"{pi}={vec.get(pi,'X')}" for pi in PIs)
    else:
        vec_str = "-"

    return {
        "Fault_Node": node,
        "Fault_Type": fault_type,
        "Detected": "YES" if ok else "NO",
        "Test_Cube": vec_str,
        "Solve_Time_s": round(et - st, 6)
    }


def run_all_faults_optimized_parallel(netlist_path, verbose=False, export_csv=True):
    PIs, POs, GATES = parse_netlist_from_file(netlist_path)
    topo_order, fanout = build_topology(PIs, POs, GATES)

    # Build fault list: Gate outputs + PIs
    fault_sites = list(GATES.keys())[:]
    for pi in PIs:
        if pi not in fault_sites:
            fault_sites.append(pi)

    # Create argument list for all faults
    jobs = []
    for node in fault_sites:
        for t in ('sa0', 'sa1'):
            jobs.append((PIs, POs, GATES, topo_order, fanout, node, t, verbose))

    print(f"\nCPU cores available: {cpu_count()}")
    print(f"Total faults to test: {len(jobs)}")

    start_all = time.time()

    # MULTIPROCESSING POOL
    with Pool(cpu_count()) as p:
        results = list(p.map(solve_single_fault, jobs))

    elapsed = time.time() - start_all

    # Convert to df
    df = pd.DataFrame(results)

    detected = df[df["Detected"]=="YES"].shape[0]
    total = df.shape[0]
    coverage = (detected / total) * 100

    print(f"\nFault Coverage: {detected}/{total} = {coverage:.2f}%")
    print(f"Total ATPG Time (parallel): {elapsed:.2f} seconds")

    # save CSV
    if export_csv:
        base = os.path.basename(netlist_path)
        name = os.path.splitext(base)[0]
        out_csv = f"{name}_podem_parallel.csv"
        df.to_csv(out_csv, index=False)
        print(f"Saved: {out_csv}")

    return df


In [ ]:
# parallel_podem.py
# Final parallel ATPG engine using ProcessPoolExecutor
# Paste this entire file and run as a script (recommended) or in a notebook cell.

import os
import time
from collections import deque, defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import pandas as pd

VERBOSE = False   # Set True to see per-fault prints from each worker (noisy)

# -----------------------
# Netlist location
# -----------------------
# Set NETLIST_PATH here or rely on uploaded_file_path variable in notebook env
if 'uploaded_file_path' in globals():
    NETLIST_PATH = uploaded_file_path
else:
    NETLIST_PATH = "uploaded_files/C17.txt"  # change as needed

if not os.path.exists(NETLIST_PATH):
    raise FileNotFoundError(f"Netlist file not found: {NETLIST_PATH}")

# -----------------------
# Parser
# -----------------------
def parse_netlist_from_file(path):
    inputs, outputs, gates = [], [], {}
    with open(path, 'r') as f:
        for raw in f:
            line = raw.strip()
            if not line or line.startswith('#'):
                continue
            if line.startswith("INPUT("):
                inputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif line.startswith("OUTPUT("):
                outputs.append(line.split("(",1)[1].split(")")[0].strip())
            elif "=" in line:
                left, right = line.split("=",1)
                gate_name = left.strip()
                typ = right.strip().split("(",1)[0].strip().lower()
                args = right.strip().split("(",1)[1].rsplit(")",1)[0]
                ins = [a.strip() for a in args.split(",") if a.strip()!=""]
                gates[gate_name] = {"type": typ, "inputs": ins}
    return inputs, outputs, gates

# -----------------------
# D-algebra helpers
# -----------------------
def logic_not(a):
    return {'0':'1','1':'0','X':'X','D':'D_BAR','D_BAR':'D'}.get(a,'X')

def logic_and_pair(a,b):
    if a=='0' or b=='0': return '0'
    if a=='1': return b
    if b=='1': return a
    if a=='X' or b=='X': return 'X'
    if a==b: return a
    if set([a,b])==set(['D','D_BAR']): return '0'
    return 'X'

def logic_or_pair(a,b):
    if a=='1' or b=='1': return '1'
    if a=='0': return b
    if b=='0': return a
    if a=='X' or b=='X': return 'X'
    if a==b: return a
    if set([a,b])==set(['D','D_BAR']): return '1'
    return 'X'

def eval_gate_D(gtype, ins_values):
    g = gtype.lower()
    if g in ('and','nand'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_and_pair(val, v)
        return logic_not(val) if g=='nand' else val
    if g in ('or','nor'):
        val = ins_values[0]
        for v in ins_values[1:]:
            val = logic_or_pair(val, v)
        return logic_not(val) if g=='nor' else val
    if g == 'not':
        return logic_not(ins_values[0])
    if g == 'buf':
        return ins_values[0]
    if g == 'xor':
        if len(ins_values) != 2: return 'X'
        a,b = ins_values
        if 'X' in (a,b): return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = ag ^ bg
        bad  = af ^ bf
        if good==bad: return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    if g == 'xnor':
        if len(ins_values) != 2: return 'X'
        a,b = ins_values
        if 'X' in (a,b): return 'X'
        def map_pair(x):
            if x=='0': return (0,0)
            if x=='1': return (1,1)
            if x=='D': return (1,0)
            if x=='D_BAR': return (0,1)
            return (None,None)
        ag,af = map_pair(a); bg,bf = map_pair(b)
        if ag is None or bg is None: return 'X'
        good = 1 - (ag ^ bg)
        bad  = 1 - (af ^ bf)
        if good==bad: return '1' if good==1 else '0'
        if good==1 and bad==0: return 'D'
        if good==0 and bad==1: return 'D_BAR'
        return 'X'
    return 'X'

# -----------------------
# topology & simulation
# -----------------------
def build_topology(PIs, POs, GATES):
    children = defaultdict(list)
    for g, info in GATES.items():
        for inp in info['inputs']:
            children[inp].append(g)
    gate_indeg = {g: sum(1 for inp in info['inputs'] if inp in GATES) for g,info in GATES.items()}
    queue = deque([g for g in GATES if gate_indeg[g] == 0])
    topo = []
    while queue:
        x = queue.popleft()
        topo.append(x)
        for ch in children.get(x, []):
            if ch in gate_indeg:
                gate_indeg[ch] -= 1
                if gate_indeg[ch] == 0:
                    queue.append(ch)
    for g in GATES:
        if g not in topo:
            topo.append(g)
    fanout = children
    return topo, fanout

def simulate_with_fault_topo(PIs, POs, GATES, topo_order, assign, fault_node=None, fault_type=None):
    vals = {pi: assign.get(pi,'X') for pi in PIs}
    # PI fault injection
    if fault_node in PIs and fault_type is not None:
        good_val = vals.get(fault_node,'X')
        if fault_type == 'sa0' and good_val == '1':
            vals[fault_node] = 'D'
        elif fault_type == 'sa1' and good_val == '0':
            vals[fault_node] = 'D_BAR'
    # one pass levelized
    for g in topo_order:
        info = GATES[g]
        ins = [vals.get(inp, 'X') for inp in info['inputs']]
        outv = eval_gate_D(info['type'], ins)
        if g == fault_node and fault_type is not None:
            gv = outv
            if fault_type == 'sa0' and gv == '1':
                outv = 'D'
            elif fault_type == 'sa1' and gv == '0':
                outv = 'D_BAR'
        vals[g] = outv
    # relaxation passes
    for _ in range(2):
        changed = False
        for g in topo_order:
            info = GATES[g]
            ins = [vals.get(inp, 'X') for inp in info['inputs']]
            outv = eval_gate_D(info['type'], ins)
            if g == fault_node and fault_type is not None:
                gv = outv
                if fault_type == 'sa0' and gv == '1':
                    outv = 'D'
                elif fault_type == 'sa1' and gv == '0':
                    outv = 'D_BAR'
            if vals.get(g,'X') != outv:
                vals[g] = outv
                changed = True
        if not changed:
            break
    return vals

# -----------------------
# X-path, backtrace, objective (same optimized logic)
# -----------------------
def x_path_exists_from(node, current_vals, fanout, GATES, POs):
    visited = set()
    stack = list(fanout.get(node, []))
    while stack:
        g = stack.pop()
        if g in visited: 
            continue
        visited.add(g)
        if g in POs:
            return True
        outv = current_vals.get(g, 'X')
        if outv == 'X':
            if any(ch in POs for ch in fanout.get(g, [])):
                return True
            stack.extend(fanout.get(g, []))
        else:
            if g in POs and outv in ('D','D_BAR'):
                return True
    return False

def non_controlling_value_for_gate(gtype):
    g = gtype.lower()
    if g in ('and','nand'): return '1'
    if g in ('or','nor'):   return '0'
    if g in ('not','buf','xor','xnor'): return None
    return None

def backtrace_to_pi_improved(node, desired_val, PIs, GATES, current_vals, fanout, POs):
    if node in PIs:
        return node, desired_val
    gate = GATES[node]
    gtype = gate['type']
    inv = gtype in ('nand','nor','not')
    pref = None
    for inp in gate['inputs']:
        if current_vals.get(inp,'X') == 'X':
            if x_path_exists_from(inp, current_vals, fanout, GATES, POs):
                pref = inp; break
    if pref is None:
        for inp in gate['inputs']:
            if current_vals.get(inp,'X') == 'X':
                pref = inp; break
    if pref is None:
        ncv = non_controlling_value_for_gate(gtype)
        if ncv is not None:
            for inp in gate['inputs']:
                if current_vals.get(inp,'X') != ncv:
                    pref = inp; break
    if pref is None:
        pref = gate['inputs'][0]
    next_val = logic_not(desired_val) if inv else desired_val
    return backtrace_to_pi_improved(pref, next_val, PIs, GATES, current_vals, fanout, POs)

def objective_improved(fault_node, fault_type, PIs, POs, GATES, current_vals, fanout):
    v_fault = current_vals.get(fault_node, 'X')
    if v_fault not in ('D','D_BAR'):
        desired = '1' if fault_type == 'sa0' else '0'
        return fault_node, desired
    candidates = []
    for g, info in GATES.items():
        outv = current_vals.get(g, 'X')
        if outv == 'X':
            in_vals = [ current_vals.get(inp,'X') for inp in info['inputs'] ]
            if any(iv in ('D','D_BAR') for iv in in_vals):
                candidates.append(g)
    for c in candidates:
        if x_path_exists_from(c, current_vals, fanout, GATES, POs):
            return c, '1'
    if candidates:
        return candidates[0], '1'
    for po in POs:
        if current_vals.get(po) in ('D','D_BAR'):
            return None, None
    return None, None

# -----------------------
# Implication with cache & PODEM search
# -----------------------
def imply_with_cache(PIs, POs, GATES, topo_order, assign, fault_node, fault_type, sim_cache):
    key = tuple(sorted((pi, val) for pi, val in assign.items() if val != 'X'))
    if key in sim_cache:
        return sim_cache[key]
    vals = simulate_with_fault_topo(PIs, POs, GATES, topo_order, assign, fault_node, fault_type)
    sim_cache[key] = vals
    return vals

def podem_optimized(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=False, max_depth=500):
    init_assign = {pi:'X' for pi in PIs}
    visited = set()
    sim_cache = {}
    def assign_key(assign):
        return tuple(sorted((pi, assign[pi]) for pi in assign if assign[pi] != 'X'))
    def search(assign, depth=0):
        if depth > max_depth: return None
        vals = imply_with_cache(PIs, POs, GATES, topo_order, assign, fault_node, fault_type, sim_cache)
        if any(vals.get(po) in ('D','D_BAR') for po in POs):
            return assign
        if all(assign[pi] in ('0','1') for pi in PIs):
            return None
        key = assign_key(assign)
        if key in visited: return None
        visited.add(key)
        obj_node, obj_val = objective_improved(fault_node, fault_type, PIs, POs, GATES, vals, fanout)
        if obj_node is None: return None
        pi, pi_val = backtrace_to_pi_improved(obj_node, obj_val, PIs, GATES, vals, fanout, POs)
        if assign[pi] in ('0','1'):
            return search(assign, depth+1)
        trial_order = [pi_val, '0' if pi_val=='1' else '1']
        for tv in trial_order:
            new_assign = assign.copy()
            new_assign[pi] = tv
            res = search(new_assign, depth+1)
            if res is not None:
                return res
        return None
    return search(init_assign, depth=0)

def run_podem_for_fault_fast(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=VERBOSE):
    vec = podem_optimized(PIs, POs, GATES, topo_order, fanout, fault_node, fault_type, verbose=verbose)
    if vec:
        if verbose:
            print(f"\n Test cube for {fault_node} {fault_type}:")
            for pi in PIs:
                print(f"  {pi}: {vec.get(pi,'X')}")
        return True, vec
    else:
        if verbose:
            print(f"\n Fault {fault_node} {fault_type} is untestable (or not found).")
        return False, None

# -----------------------
# Worker: process a chunk of faults
# -----------------------
def process_chunk(args):
    # args: (PIs, POs, GATES, topo_order, fanout, fault_chunk, verbose)
    PIs, POs, GATES, topo_order, fanout, fault_chunk, verbose = args
    local_results = []
    for node, fault_type in fault_chunk:
        st = time.time()
        ok, vec = run_podem_for_fault_fast(PIs, POs, GATES, topo_order, fanout, node, fault_type, verbose=verbose)
        et = time.time()
        if ok and vec:
            vec_str = " ".join(f"{pi}={vec.get(pi,'X')}" for pi in PIs)
        else:
            vec_str = "-"
        local_results.append({
            "Fault_Node": node,
            "Fault_Type": fault_type,
            "Detected": "YES" if ok else "NO",
            "Test_Cube": vec_str,
            "Solve_Time_s": round(et - st, 6)
        })
    return local_results

# -----------------------
# Parallel driver
# -----------------------
def split_into_chunks(items, n):
    # distribute nearly equally
    chunks = [[] for _ in range(n)]
    for i, item in enumerate(items):
        chunks[i % n].append(item)
    return [c for c in chunks if c]

def run_all_faults_parallel(netlist_path, verbose=False, export_csv=True, max_workers=None):
    PIs, POs, GATES = parse_netlist_from_file(netlist_path)
    topo_order, fanout = build_topology(PIs, POs, GATES)

    # build fault list: gates + PIs (unique)
    fault_sites = list(GATES.keys())[:]
    for pi in PIs:
        if pi not in fault_sites:
            fault_sites.append(pi)
    # make list of (node, fault_type)
    all_faults = []
    for node in fault_sites:
        all_faults.append((node, 'sa0'))
        all_faults.append((node, 'sa1'))

    if max_workers is None:
        max_workers = max(1, multiprocessing.cpu_count())

    # split into chunks equal to workers
    chunks = split_into_chunks(all_faults, max_workers)
    job_args = []
    for chunk in chunks:
        job_args.append((PIs, POs, GATES, topo_order, fanout, chunk, verbose))

    print(f"CPU cores used: {len(chunks)}")
    print(f"Total faults to test: {len(all_faults)}")
    t_start = time.time()

    results = []
    # Use ProcessPoolExecutor
    with ProcessPoolExecutor(max_workers=len(chunks)) as ex:
        futures = [ex.submit(process_chunk, arg) for arg in job_args]
        for fut in as_completed(futures):
            res = fut.result()
            results.extend(res)

    elapsed = time.time() - t_start
    df = pd.DataFrame(results)
    total = len(results)
    detected = df[df["Detected"]=="YES"].shape[0]
    coverage = (detected / total) * 100 if total>0 else 0.0

    print(f"\nFault Coverage: {detected}/{total} = {coverage:.2f}%")
    print(f"Total ATPG Time (parallel): {elapsed:.2f} seconds")

    if export_csv:
        base = os.path.basename(netlist_path)
        name = os.path.splitext(base)[0]
        out_csv = f"{name}_podem_parallel.csv"
        df.to_csv(out_csv, index=False)
        print(f"Saved: {out_csv}")

    return df

# -----------------------
# Run when executed as script
# -----------------------
if __name__ == "__main__":
    # recommended: run from terminal for best multiprocessing behavior
    print("Using netlist:", NETLIST_PATH)
    # adjust max_workers if you want to limit cores, otherwise uses all cores
    df_cov = run_all_faults_parallel(NETLIST_PATH, verbose=VERBOSE, export_csv=True, max_workers=None)
    print("\nDone.")


Using netlist: uploaded_files\C432.txt
CPU cores used: 12
Total faults to test: 392
